In [1]:
import mysql.connector
from flask import Flask, request, jsonify

app = Flask(__name__)

# Configurar conexão com o banco de dados MySQL
db_config = {
    'user': 'root',
    'password': 'your_password',
    'host': '127.0.0.1',
    'database': 'financing_db'
}

def execute_stored_procedure(proc_name, params):
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()
    cursor.callproc(proc_name, params)
    results = [result.fetchall() for result in cursor.stored_results()]
    cursor.close()
    conn.close()
    return results

@app.route('/calculate_financing', methods=['POST'])
def calculate_financing():
    data = request.get_json()
    property_value = data['property_value']
    down_payment = data['down_payment']
    annual_interest_rate = data['annual_interest_rate']
    term_years = data['term_years']

    financed_amount = property_value - down_payment

    # Call the stored procedures
    monthly_payment_result = execute_stored_procedure('calculate_monthly_payment', (financed_amount, annual_interest_rate, term_years, 0))
    monthly_payment = monthly_payment_result[0][0][0]

    total_payment_result = execute_stored_procedure('calculate_total_payment', (monthly_payment, term_years, 0))
    total_payment = total_payment_result[0][0][0]

    # Save the results in the database
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO financing (property_value, down_payment, financed_amount, interest_rate, term_years, monthly_payment, total_payment)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (property_value, down_payment, financed_amount, annual_interest_rate, term_years, monthly_payment, total_payment))
    conn.commit()
    cursor.close()
    conn.close()

    return jsonify({
        'monthly_payment': monthly_payment,
        'total_payment': total_payment
    })

if __name__ == '__main__':
    app.run(debug=True)


ModuleNotFoundError: No module named 'mysql'